Report for Task A based on Deep Neural Network using H2O and python.


This imports the h2o library and connects with the h2o server. The server has to be running.

In [1]:
import h2o

# switch to java 8 "sdk use java 8.0.452-tem"
# java -jar h2o.jar
h2o.init(ip="localhost", port="54321")

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,22 secs
H2O_cluster_timezone:,Europe/Belgrade
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,1 month and 28 days
H2O_cluster_name:,adama
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.988 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Helper function to build and test deep neural network model

In [2]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

# def run_experiment(dataset, features, target, hidden, nfolds, activation):
#     train, test = dataset.split_frame(ratios=[0.8])
#     dnn = H2ODeepLearningEstimator(activation=activation, hidden=hidden, nfolds=nfolds)
#     dnn.train(x=features, y=target, training_frame=train, validation_frame=test)
#     print(dnn.show())
mean_error_metrics = []
mse_metrics = []
rmse_metrics = []

def run_experiment(dataset, features, target, hidden, nfolds, activation):
    train, test = dataset.split_frame(ratios=[0.8], seed=42)
    dnn = H2ODeepLearningEstimator(activation=activation, hidden=hidden, nfolds=nfolds, seed=42, reproducible=True)
    dnn.train(x=features, y=target, training_frame=train, validation_frame=test)
    #print(dnn.mean_per_class_error(xval=True))
    #print(dnn.run_time / 1000)
    # print(type(dnn))
    # print(dnn.mse())
    # print(dnn.rmse())
    print(dnn.show())
    mean_error_metrics.append([activation, "None" if nfolds == 0 else f"{nfolds}-folds", hidden, 
                    f"{dnn.mean_per_class_error(train=True) * 100}%", 
                    f"{dnn.mean_per_class_error(valid=True) * 100}%", 
                    f"{dnn.mean_per_class_error(xval=True) * 100}%" if nfolds != 0 else "-", 
                    f"{dnn.run_time / 1000} s"])
    mse_metrics.append([activation, "None" if nfolds == 0 else f"{nfolds}-folds", hidden, 
                    f"{dnn.mse(train=True)}", 
                    f"{dnn.mse(valid=True)}",
                    f"{dnn.mse(xval=True)}" if nfolds != 0 else "-"])
    rmse_metrics.append([activation, "None" if nfolds == 0 else f"{nfolds}-folds", hidden, 
                    f"{dnn.rmse(train=True)}", 
                    f"{dnn.rmse(valid=True)}",
                    f"{dnn.rmse(xval=True)}" if nfolds != 0 else "-"])

Iris dataset

In [4]:
col_names = ["party", "handicapped-infants", "water-project-cost-sharing", "adoption-of-the-budget-resolution", "physician-fee-freeze", "el-salvador-aid", "religious-groups-in-schools", "anti-satellite-test-ban", "aid-to-nicaraguan-contras", "mx-missile", "immigration", "synfuels-corporation-cutback", "education-spending", "superfund-right-to-sue", "crime", "duty-free-exports", "export-administration-act-south-africa"]
votes = h2o.import_file(path="datasets/votes/house-votes-84.data", col_names=col_names)

features = col_names[1:]
target = "party"

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,], cross folds = 0, activation function = "rectifier"

In [5]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=0, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_1


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 397 weights/biases, 15.7 KB, 3,380 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight             weight_rms           mean_bias                bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ----------------------  -------------------  -----------------------  -------------------
    1        64       Input      0.0
    2        5        Rectifier  0.0        0.0   0.0   0.26967561226956605    0.4383763074874878     0.0         -0.0063685370201710615  0.181473970413208    0.5651113056292081       0.06373605132102966
    3        5        Rectifier  0.0        0.0   0.0   0.00854968486353755    0.0160413458943367     0.0         -0.06654038399457932    0.4950789213180542   0.9434190147898665       0.07188418507575989
    4        5        Rectifier  0.0        0.0   0.0   0.005699532681610435   0.008003383874893188   0.0         0.06221109122037888     0.40610623359680176  1.1627540761348671       0.6661279201507568
    5        2        Softmax               0.0   0.0   0.0015363607555627823  0.0003694383194670081  0.0         1.6811179116368293      1.0909318923950195   -1.3877787807814457e-17  0.16258132457733154

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.02484122938261853
RMSE: 0.1576110065402113
LogLoss: 0.09519415971222477
Mean Per-Class Error: 0.024566048837893498
AUC: 0.9920197116799058
AUCPR: 0.979104947040616
Gini: 0.9840394233598115

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.47264731686597217
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    199         7             0.034    (7.0/206.0)
republican  2           130           0.0152   (2.0/132.0)
Total       201         137           0.0266   (9.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.472647     0.966543  66
max f2                       0.472647     0.977444  66
max f0point5                 0.713803     0.966258  59
max accuracy                 0.613505     0.973373  64
max precision                0.947205     0.990099  32
max recall                   0.0624061    1         89
max specificity              0.97684      0.995146  0
max absolute_mcc             0.472647     0.944893  66
max min_per_class_accuracy   0.613505     0.970874  64
max mean_per_class_accuracy  0.472647     0.975434  66
max tns                      0.97684      205       0
max fns                      0.97684      82        0
max fps                      3.33231e-06  206       237
max tps                      0.0624061    132       89
max tnr                      0.97684      0.995146  0
max fnr                      0.97684      0.621212  0
max fpr                      3.33231e-06  1         237
max tpr                      0.0624061    1         89

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 39.11 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.150888                    0.97684            2.5104    2.5104       

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 0, activation function = "rectifier"


In [6]:
run_experiment(dataset=votes, features=features, target=target, hidden=[20,20,20], nfolds=0, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_22


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 2,182 weights/biases, 37.2 KB, 3,380 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight           weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  --------------------  ----------  --------------------  -------------------  ----------------------  --------------------
    1        64       Input      0.0
    2        20       Rectifier  0.0        0.0   0.0   0.2607023372629101    0.4345998764038086    0.0         0.000635204364607489  0.15902632474899292  0.5068950222268198      0.0374615341424942
    3        20       Rectifier  0.0        0.0   0.0   0.004254248169454513  0.012103579938411713  0.0         0.004524674203203176  0.22199571132659912  1.0009818604902923      0.025309868156909943
    4        20       Rectifier  0.0        0.0   0.0   0.052450510103553824  0.21675407886505127   0.0         0.009121749889745843  0.2140645980834961   0.9964017500557858      0.018942423164844513
    5        2        Softmax               0.0   0.0   0.05359956882020924   0.2165442705154419    0.0         0.03699527545832097   1.1526083946228027   -6.938893903907228e-18  0.023463621735572815

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.014514370855168185
RMSE: 0.12047560273834776
LogLoss: 0.058711462923311385
Mean Per-Class Error: 0.01379082082965578
AUC: 0.9970947337452192
AUCPR: 0.9939040192884351
Gini: 0.9941894674904383

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.8375846872799153
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    205         1             0.0049   (1.0/206.0)
republican  3           129           0.0227   (3.0/132.0)
Total       208         130           0.0118   (4.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.837585     0.984733  91
max f2                       0.470696     0.990991  99
max f0point5                 0.837585     0.989264  91
max accuracy                 0.837585     0.988166  91
max precision                0.999039     1         0
max recall                   0.470696     1         99
max specificity              0.999039     1         0
max absolute_mcc             0.837585     0.975148  91
max min_per_class_accuracy   0.782241     0.984848  94
max mean_per_class_accuracy  0.621781     0.986503  96
max tns                      0.999039     206       0
max fns                      0.999039     131       0
max fps                      3.32293e-07  206       269
max tps                      0.470696     132       99
max tnr                      0.999039     1         0
max fnr                      0.999039     0.992424  0
max fpr                      3.32293e-07  1         269
max tpr                      0.470696     1         99

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 40.39 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0177515                   0.998903           2.56061    2.56061            1            

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 0, activation function = "rectifier"

In [7]:
run_experiment(dataset=votes, features=features, target=target, hidden=[100,100,100], nfolds=0, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_43


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 26,902 weights/biases, 329.7 KB, 3,380 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight            weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  --------------------  ----------  ---------------------  -------------------  ----------------------  ---------------------
    1        64       Input      0.0
    2        100      Rectifier  0.0        0.0   0.0   0.25351739970712517   0.43067610263824463   0.0         0.0022138433337661923  0.10979557037353516  0.4916553172481135      0.01633206009864807
    3        100      Rectifier  0.0        0.0   0.0   0.006305315609421814  0.006432496011257172  0.0         -0.00278441449620816   0.10121926665306091  0.9928183312808858      0.007723288610577583
    4        100      Rectifier  0.0        0.0   0.0   0.09900077089754632   0.2692042589187622    0.0         0.0004197641423509353  0.10057049989700317  0.9993520426318344      0.001850338652729988
    5        2        Softmax               0.0   0.0   0.09241412103234324   0.2819918394088745    0.0         -0.007474530169274658  0.5522544384002686   1.4941661189565858e-18  5.129433702677488e-05

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.021223530932150892
RMSE: 0.14568298092828447
LogLoss: 0.0788892250668668
Mean Per-Class Error: 0.019711679905854665
AUC: 0.9974992644895558
AUCPR: 0.9959878357501917
Gini: 0.9949985289791117

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.8012458466893759
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    201         5             0.0243   (5.0/206.0)
republican  2           130           0.0152   (2.0/132.0)
Total       203         135           0.0207   (7.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.801246     0.973783  96
max f2                       0.122421     0.980684  106
max f0point5                 0.973124     0.982704  87
max accuracy                 0.91014      0.97929   94
max precision                1            1         0
max recall                   0.122421     1         106
max specificity              1            1         0
max absolute_mcc             0.801246     0.956836  96
max min_per_class_accuracy   0.91014      0.977273  94
max mean_per_class_accuracy  0.801246     0.980288  96
max tns                      1            206       0
max fns                      1            131       0
max fps                      1.00753e-08  206       269
max tps                      0.122421     132       106
max tnr                      1            1         0
max fnr                      1            0.992424  0
max fpr                      1.00753e-08  1         269
max tpr                      0.122421     1         106

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 40.77 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0118343                   0.999999           2.56061   2.56061            1 

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 5, activation function = "rectifier"


In [8]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=5, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_66


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 397 weights/biases, 15.7 KB, 3,718 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms                momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  ----------------------  ----------  ---------------------  -------------------  -----------------------  -------------------
    1        64       Input      0.0
    2        5        Rectifier  0.0        0.0   0.0   0.26967864367388755   0.43837499618530273     0.0         -0.006552608267702454  0.18194323778152466  0.5640372000500701       0.06298166513442993
    3        5        Rectifier  0.0        0.0   0.0   0.008621125557110644  0.016292452812194824    0.0         -0.06637037634849548   0.49541640281677246  0.9437223901522694       0.07295086979866028
    4        5        Rectifier  0.0        0.0   0.0   0.005201225398341194  0.007149899378418922    0.0         0.06249721698462963    0.40761470794677734  1.1628881981617805       0.666752815246582
    5        2        Softmax               0.0   0.0   0.001456583570688963  0.00034134811721742153  0.0         1.681117482483387      1.093127727508545    -2.7755575615628914e-17  0.1634746789932251

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.02357065678931948
RMSE: 0.15352738123644094
LogLoss: 0.09121775368549542
Mean Per-Class Error: 0.02213886437187408
AUC: 0.9923323036187113
AUCPR: 0.9789850679136286
Gini: 0.9846646072374226

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4651299332629031
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    200         6             0.0291   (6.0/206.0)
republican  2           130           0.0152   (2.0/132.0)
Total       202         136           0.0237   (8.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.46513      0.970149  67
max f2                       0.46513      0.978916  67
max f0point5                 0.610821     0.969697  63
max accuracy                 0.610821     0.976331  63
max precision                0.913478     0.990291  36
max recall                   0.0551605    1         91
max specificity              0.977566     0.995146  0
max absolute_mcc             0.46513      0.950839  67
max min_per_class_accuracy   0.536815     0.975728  65
max mean_per_class_accuracy  0.46513      0.977861  67
max tns                      0.977566     205       0
max fns                      0.977566     87        0
max fps                      2.562e-06    206       239
max tps                      0.0551605    132       91
max tnr                      0.977566     0.995146  0
max fnr                      0.977566     0.659091  0
max fpr                      2.562e-06    1         239
max tpr                      0.0551605    1         91

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 38.40 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.136095                    0.977566           2.50494   2.50494            0

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 5, activation function = "rectifier"


In [9]:
run_experiment(dataset=votes, features=features, target=target, hidden=[20,20,20], nfolds=5, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_229


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 2,182 weights/biases, 37.2 KB, 3,718 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms             momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  -------------------  ----------  ---------------------  -------------------  -----------------------  --------------------
    1        64       Input      0.0
    2        20       Rectifier  0.0        0.0   0.0   0.26074486959576004   0.4345804452896118   0.0         0.0005905363869032953  0.15911906957626343  0.506591416448452        0.03704230487346649
    3        20       Rectifier  0.0        0.0   0.0   0.004276484863075893  0.0121263787150383   0.0         0.004264391815522686   0.22209841012954712  1.0005178812546662       0.02478756010532379
    4        20       Rectifier  0.0        0.0   0.0   0.05253426652980124   0.21673959493637085  0.0         0.00931727505492745    0.2141045331954956   0.9965031925472785       0.018423162400722504
    5        2        Softmax               0.0   0.0   0.05367538485006662   0.21653014421463013  0.0         0.036995189543813464   1.1533446311950684   -1.0408340855860843e-17  0.023069165647029877

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.012480409338631761
RMSE: 0.1117157524193959
LogLoss: 0.05231274762045036
Mean Per-Class Error: 0.012135922330097087
AUC: 0.9973153868784936
AUCPR: 0.9946770169199727
Gini: 0.9946307737569873

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3643252864977982
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    201         5             0.0243   (5.0/206.0)
republican  0           132           0        (0.0/132.0)
Total       201         137           0.0148   (5.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.364325     0.981413  98
max f2                       0.364325     0.992481  98
max f0point5                 0.751891     0.986025  89
max accuracy                 0.660183     0.985207  94
max precision                0.998695     1         0
max recall                   0.364325     1         98
max specificity              0.998695     1         0
max absolute_mcc             0.364325     0.969597  98
max min_per_class_accuracy   0.660183     0.984848  94
max mean_per_class_accuracy  0.364325     0.987864  98
max tns                      0.998695     206       0
max fns                      0.998695     131       0
max fps                      9.94648e-08  206       269
max tps                      0.364325     132       98
max tnr                      0.998695     1         0
max fnr                      0.998695     0.992424  0
max fpr                      9.94648e-08  1         269
max tpr                      0.364325     1         98

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 39.25 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0177515                   0.99866            2.56061    2.56061            1      

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 5, activation function = "rectifier"

In [10]:
run_experiment(dataset=votes, features=features, target=target, hidden=[100,100,100], nfolds=5, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_398


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 26,902 weights/biases, 329.7 KB, 3,380 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  --------------------  ----------  ----------------------  -------------------  ----------------------  ---------------------
    1        64       Input      0.0
    2        100      Rectifier  0.0        0.0   0.0   0.25317598584423195   0.43078601360321045   0.0         0.001498042455518771    0.11016631126403809  0.48586404208152467     0.023875869810581207
    3        100      Rectifier  0.0        0.0   0.0   0.006349660054181004  0.008466988801956177  0.0         -0.00529289683494253    0.10148590803146362  0.9869213140279145      0.011644832789897919
    4        100      Rectifier  0.0        0.0   0.0   0.07301987967599416   0.21971482038497925   0.0         -0.0002623463712330704  0.10068872570991516  0.998461964173011       0.0033484846353530884
    5        2        Softmax               0.0   0.0   0.06799390562751796   0.2369123101234436    0.0         -0.007474518812960014   0.5512197017669678   3.2526065174565133e-18  0.0005023693665862083

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.037598742595516474
RMSE: 0.19390395198529728
LogLoss: 0.14963630711691328
Mean Per-Class Error: 0.009708737864077669
AUC: 0.9991909385113269
AUCPR: 0.9987273854685967
Gini: 0.9983818770226538

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9625960973110883
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    202         4             0.0194   (4.0/206.0)
republican  0           132           0        (0.0/132.0)
Total       202         136           0.0118   (4.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.962596     0.985075  97
max f2                       0.962596     0.993976  97
max f0point5                 0.99962      0.985577  84
max accuracy                 0.994869     0.988166  95
max precision                1            1         0
max recall                   0.962596     1         97
max specificity              1            1         0
max absolute_mcc             0.962596     0.975573  97
max min_per_class_accuracy   0.994869     0.985437  95
max mean_per_class_accuracy  0.962596     0.990291  97
max tns                      1            206       0
max fns                      1            131       0
max fps                      9.99608e-10  206       269
max tps                      0.962596     132       97
max tnr                      1            1         0
max fnr                      1            0.992424  0
max fpr                      9.99608e-10  1         269
max tpr                      0.962596     1         97

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 43.81 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0118343                   1                  2.56061    2.56061     

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 10, activation function = "rectifier"


In [11]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=10, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_555


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 397 weights/biases, 15.7 KB, 3,718 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms                momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  ----------------------  ----------  ---------------------  -------------------  -----------------------  -------------------
    1        64       Input      0.0
    2        5        Rectifier  0.0        0.0   0.0   0.26967864367388755   0.43837499618530273     0.0         -0.006552608267702454  0.18194323778152466  0.5640372000500701       0.06298166513442993
    3        5        Rectifier  0.0        0.0   0.0   0.008621125557110644  0.016292452812194824    0.0         -0.06637037634849548   0.49541640281677246  0.9437223901522694       0.07295086979866028
    4        5        Rectifier  0.0        0.0   0.0   0.005201225398341194  0.007149899378418922    0.0         0.06249721698462963    0.40761470794677734  1.1628881981617805       0.666752815246582
    5        2        Softmax               0.0   0.0   0.001456583570688963  0.00034134811721742153  0.0         1.681117482483387      1.093127727508545    -2.7755575615628914e-17  0.1634746789932251

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.02357065678931948
RMSE: 0.15352738123644094
LogLoss: 0.09121775368549542
Mean Per-Class Error: 0.02213886437187408
AUC: 0.9923323036187113
AUCPR: 0.9789850679136286
Gini: 0.9846646072374226

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4651299332629031
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    200         6             0.0291   (6.0/206.0)
republican  2           130           0.0152   (2.0/132.0)
Total       202         136           0.0237   (8.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.46513      0.970149  67
max f2                       0.46513      0.978916  67
max f0point5                 0.610821     0.969697  63
max accuracy                 0.610821     0.976331  63
max precision                0.913478     0.990291  36
max recall                   0.0551605    1         91
max specificity              0.977566     0.995146  0
max absolute_mcc             0.46513      0.950839  67
max min_per_class_accuracy   0.536815     0.975728  65
max mean_per_class_accuracy  0.46513      0.977861  67
max tns                      0.977566     205       0
max fns                      0.977566     87        0
max fps                      2.562e-06    206       239
max tps                      0.0551605    132       91
max tnr                      0.977566     0.995146  0
max fnr                      0.977566     0.659091  0
max fpr                      2.562e-06    1         239
max tpr                      0.0551605    1         91

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 38.40 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.136095                    0.977566           2.50494   2.50494            

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 10, activation function = "rectifier"

In [12]:
run_experiment(dataset=votes, features=features, target=target, hidden=[20,20,20], nfolds=10, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_829


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 2,182 weights/biases, 37.2 KB, 3,718 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms             momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  -------------------  ----------  ---------------------  -------------------  -----------------------  --------------------
    1        64       Input      0.0
    2        20       Rectifier  0.0        0.0   0.0   0.26074486959576004   0.4345804452896118   0.0         0.0005905363869032953  0.15911906957626343  0.506591416448452        0.03704230487346649
    3        20       Rectifier  0.0        0.0   0.0   0.004276484863075893  0.0121263787150383   0.0         0.004264391815522686   0.22209841012954712  1.0005178812546662       0.02478756010532379
    4        20       Rectifier  0.0        0.0   0.0   0.05253426652980124   0.21673959493637085  0.0         0.00931727505492745    0.2141045331954956   0.9965031925472785       0.018423162400722504
    5        2        Softmax               0.0   0.0   0.05367538485006662   0.21653014421463013  0.0         0.036995189543813464   1.1533446311950684   -1.0408340855860843e-17  0.023069165647029877

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.012480409338631761
RMSE: 0.1117157524193959
LogLoss: 0.05231274762045036
Mean Per-Class Error: 0.012135922330097087
AUC: 0.9973153868784936
AUCPR: 0.9946770169199727
Gini: 0.9946307737569873

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3643252864977982
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    201         5             0.0243   (5.0/206.0)
republican  0           132           0        (0.0/132.0)
Total       201         137           0.0148   (5.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.364325     0.981413  98
max f2                       0.364325     0.992481  98
max f0point5                 0.751891     0.986025  89
max accuracy                 0.660183     0.985207  94
max precision                0.998695     1         0
max recall                   0.364325     1         98
max specificity              0.998695     1         0
max absolute_mcc             0.364325     0.969597  98
max min_per_class_accuracy   0.660183     0.984848  94
max mean_per_class_accuracy  0.364325     0.987864  98
max tns                      0.998695     206       0
max fns                      0.998695     131       0
max fps                      9.94648e-08  206       269
max tps                      0.364325     132       98
max tnr                      0.998695     1         0
max fnr                      0.998695     0.992424  0
max fpr                      9.94648e-08  1         269
max tpr                      0.364325     1         98

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 39.25 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0177515                   0.99866            2.56061    2.56061            1      

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 10, activation function = "rectifier"


In [13]:
run_experiment(dataset=votes, features=features, target=target, hidden=[100,100,100], nfolds=10, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_1105


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 26,902 weights/biases, 329.7 KB, 3,718 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight             weight_rms           mean_bias              bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  --------------------  ----------  ----------------------  -------------------  ---------------------  ----------------------
    1        64       Input      0.0
    2        100      Rectifier  0.0        0.0   0.0   0.25330591148541315   0.43072259426116943   0.0         0.0014377941788416137   0.11018452048301697  0.4852396061028154     0.024163298308849335
    3        100      Rectifier  0.0        0.0   0.0   0.007274869144178228  0.010717805474996567  0.0         -0.005477823818532374   0.10150739550590515  0.9864373116969942     0.01211453229188919
    4        100      Rectifier  0.0        0.0   0.0   0.07226387432205957   0.21583563089370728   0.0         -0.0004305749288701918  0.10070693492889404  0.9982090311574716     0.0034842360764741898
    5        2        Softmax               0.0   0.0   0.06796705719374586   0.23631423711776733   0.0         -0.007474514635978266   0.5510728359222412   2.778268066994105e-18  0.00016308255726471543

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.010393406739185063
RMSE: 0.10194805902607985
LogLoss: 0.03078602815911163
Mean Per-Class Error: 0.008642247719917623
AUC: 0.9993380406001766
AUCPR: 0.9989627086845767
Gini: 0.9986760812003532

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.35701591035352204
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    204         2             0.0097   (2.0/206.0)
republican  1           131           0.0076   (1.0/132.0)
Total       205         133           0.0089   (3.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.357016     0.988679  94
max f2                       0.166649     0.992481  98
max f0point5                 0.794117     0.987261  85
max accuracy                 0.357016     0.991124  94
max precision                1            1         0
max recall                   0.166649     1         98
max specificity              1            1         0
max absolute_mcc             0.357016     0.981399  94
max min_per_class_accuracy   0.357016     0.990291  94
max mean_per_class_accuracy  0.357016     0.991358  94
max tns                      1            206       0
max fns                      1            131       0
max fps                      7.90727e-12  206       269
max tps                      0.166649     132       98
max tnr                      1            1         0
max fnr                      1            0.992424  0
max fpr                      7.90727e-12  1         269
max tpr                      0.166649     1         98

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 38.42 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0118343                   1                  2.56061    2.56061        

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 0, activation function = "tanh"


In [14]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=0, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_1385


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 397 weights/biases, 15.7 KB, 3,380 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias               bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  ----------------------  --------------------
    1        64       Input    0.0
    2        5        Tanh     0.0        0.0   0.0   0.25244694898956366    0.43114256858825684    0.0         -0.014898876845973064  0.1780187487602234   -3.0877236202137e-05    0.1196337342262268
    3        5        Tanh     0.0        0.0   0.0   0.0020506446668878196  0.0007595927454531193  0.0         -0.06091129034757614   0.48763978481292725  0.064163481603456       0.030520908534526825
    4        5        Tanh     0.0        0.0   0.0   0.012981761889532208   0.01557065173983574    0.0         0.16969490901799872    0.3945540189743042   0.056681598694319955    0.027812950313091278
    5        2        Softmax             0.0   0.0   0.003327851835638285   0.0006053755059838295  0.0         1.6811198571696877     1.0460100173950195   1.1102230246251565e-16  0.09112781286239624

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.01548889297470334
RMSE: 0.12445438109887229
LogLoss: 0.0691635521869766
Mean Per-Class Error: 0.014857310973815829
AUC: 0.995660488378935
AUCPR: 0.9940360901404862
Gini: 0.9913209767578699

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5700888974598675
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    203         3             0.0146   (3.0/206.0)
republican  2           130           0.0152   (2.0/132.0)
Total       205         133           0.0148   (5.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.570089     0.981132  94
max f2                       0.46862      0.986446  97
max f0point5                 0.680311     0.981595  91
max accuracy                 0.570089     0.985207  94
max precision                0.994673     1         0
max recall                   0.00582885   1         158
max specificity              0.994673     1         0
max absolute_mcc             0.570089     0.968986  94
max min_per_class_accuracy   0.570089     0.984848  94
max mean_per_class_accuracy  0.570089     0.985143  94
max tns                      0.994673     206       0
max fns                      0.994673     131       0
max fps                      0.00251287   206       269
max tps                      0.00582885   132       158
max tnr                      0.994673     1         0
max fnr                      0.994673     0.992424  0
max fpr                      0.00251287   1         269
max tpr                      0.00582885   1         158

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 39.13 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0118343                   0.993762           2.56061    2.56061            1           

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 0, activation function = "tanh"


In [15]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=0, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_1406


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 397 weights/biases, 15.7 KB, 3,380 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias               bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  ----------------------  --------------------
    1        64       Input    0.0
    2        5        Tanh     0.0        0.0   0.0   0.25244694898956366    0.43114256858825684    0.0         -0.014898876845973064  0.1780187487602234   -3.0877236202137e-05    0.1196337342262268
    3        5        Tanh     0.0        0.0   0.0   0.0020506446668878196  0.0007595927454531193  0.0         -0.06091129034757614   0.48763978481292725  0.064163481603456       0.030520908534526825
    4        5        Tanh     0.0        0.0   0.0   0.012981761889532208   0.01557065173983574    0.0         0.16969490901799872    0.3945540189743042   0.056681598694319955    0.027812950313091278
    5        2        Softmax             0.0   0.0   0.003327851835638285   0.0006053755059838295  0.0         1.6811198571696877     1.0460100173950195   1.1102230246251565e-16  0.09112781286239624

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.01548889297470334
RMSE: 0.12445438109887229
LogLoss: 0.0691635521869766
Mean Per-Class Error: 0.014857310973815829
AUC: 0.995660488378935
AUCPR: 0.9940360901404862
Gini: 0.9913209767578699

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5700888974598675
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    203         3             0.0146   (3.0/206.0)
republican  2           130           0.0152   (2.0/132.0)
Total       205         133           0.0148   (5.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.570089     0.981132  94
max f2                       0.46862      0.986446  97
max f0point5                 0.680311     0.981595  91
max accuracy                 0.570089     0.985207  94
max precision                0.994673     1         0
max recall                   0.00582885   1         158
max specificity              0.994673     1         0
max absolute_mcc             0.570089     0.968986  94
max min_per_class_accuracy   0.570089     0.984848  94
max mean_per_class_accuracy  0.570089     0.985143  94
max tns                      0.994673     206       0
max fns                      0.994673     131       0
max fps                      0.00251287   206       269
max tps                      0.00582885   132       158
max tnr                      0.994673     1         0
max fnr                      0.994673     0.992424  0
max fpr                      0.00251287   1         269
max tpr                      0.00582885   1         158

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 39.13 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0118343                   0.993762           2.56061    2.56061            1           

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 0, activation function = "tanh"


In [16]:
run_experiment(dataset=votes, features=features, target=target, hidden=[20,20,20], nfolds=0, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_1427


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 2,182 weights/biases, 37.2 KB, 3,380 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  ---------------------  ----------  ----------------------  -------------------  ----------------------  --------------------
    1        64       Input    0.0
    2        20       Tanh     0.0        0.0   0.0   0.25288375200802876   0.4309276342391968     0.0         -0.0006292716017185285  0.15804606676101685  -0.0032268937639498814  0.02844873070716858
    3        20       Tanh     0.0        0.0   0.0   0.003910838651281665  0.0033601103350520134  0.0         0.00511955373629462     0.2218475341796875   0.007948761646788038    0.023779861629009247
    4        20       Tanh     0.0        0.0   0.0   0.004441358810872771  0.003535749390721321   0.0         0.01554943269395153     0.21376240253448486  0.0035175324453470395   0.011715516448020935
    5        2        Softmax             0.0   0.0   0.00474102896405384   0.0014152303338050842  0.0         0.03699496034532786     1.1523737907409668   -5.204170427930421e-18  0.006956985220313072

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.015894808045215335
RMSE: 0.12607461300839012
LogLoss: 0.05654426607172878
Mean Per-Class Error: 0.018645189761694617
AUC: 0.9979773462783172
AUCPR: 0.9967656423884548
Gini: 0.9959546925566345

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6693947349504704
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    203         3             0.0146   (3.0/206.0)
republican  3           129           0.0227   (3.0/132.0)
Total       206         132           0.0178   (6.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.669395     0.977273  93
max f2                       0.264208     0.983607  104
max f0point5                 0.861033     0.979299  85
max accuracy                 0.669395     0.982249  93
max precision                0.999944     1         0
max recall                   0.264208     1         104
max specificity              0.999944     1         0
max absolute_mcc             0.669395     0.96271   93
max min_per_class_accuracy   0.669395     0.977273  93
max mean_per_class_accuracy  0.669395     0.981355  93
max tns                      0.999944     206       0
max fns                      0.999944     131       0
max fps                      1.95182e-05  206       269
max tps                      0.264208     132       104
max tnr                      0.999944     1         0
max fnr                      0.999944     0.992424  0
max fpr                      1.95182e-05  1         269
max tpr                      0.264208     1         104

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 39.65 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0147929                   0.999861           2.56061   2.56061            1    

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 0, activation function = "tanh"


In [17]:
run_experiment(dataset=votes, features=features, target=target, hidden=[100,100,100], nfolds=0, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_1450


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 26,902 weights/biases, 329.7 KB, 3,380 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight             weight_rms           mean_bias                bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  --------------------  ----------  ----------------------  -------------------  -----------------------  ---------------------
    1        64       Input    0.0
    2        100      Tanh     0.0        0.0   0.0   0.25470612603076914    0.4300272464752197    0.0         0.0029735185033368337   0.10943356156349182  -0.0006237825568776558   0.006165862083435059
    3        100      Tanh     0.0        0.0   0.0   0.011861960638593882   0.012387227267026901  0.0         0.0005503674001993204   0.10098439455032349  -0.00047674686738618757  0.004859739914536476
    4        100      Tanh     0.0        0.0   0.0   0.027578046414162963   0.053109392523765564  0.0         0.0010802546546707162   0.10057023167610168  0.0002810132091452572    0.0018700412474572659
    5        2        Softmax             0.0   0.0   0.0044754409953020515  0.001604023389518261  0.0         -0.0074745364696718756  0.5535500049591064   -5.800481622797449e-18   0.0008556435350328684

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.022487782171216938
RMSE: 0.14995926837383855
LogLoss: 0.07514393643645458
Mean Per-Class Error: 0.02486025301559282
AUC: 0.9970947337452192
AUCPR: 0.9955915684327467
Gini: 0.9941894674904383

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6304233638481587
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    202         4             0.0194   (4.0/206.0)
republican  4           128           0.0303   (4.0/132.0)
Total       206         132           0.0237   (8.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.630423     0.969697  93
max f2                       0.404092     0.975976  99
max f0point5                 0.909297     0.975806  83
max accuracy                 0.630423     0.976331  93
max precision                0.999851     1         0
max recall                   0.109858     1         110
max specificity              0.999851     1         0
max absolute_mcc             0.630423     0.950279  93
max min_per_class_accuracy   0.451228     0.970874  96
max mean_per_class_accuracy  0.630423     0.97514   93
max tns                      0.999851     206       0
max fns                      0.999851     131       0
max fps                      2.33911e-05  206       269
max tps                      0.109858     132       110
max tnr                      0.999851     1         0
max fnr                      0.999851     0.992424  0
max fpr                      2.33911e-05  1         269
max tpr                      0.109858     1         110

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 40.10 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0177515                   0.999611           2.56061   2.56061       

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 5, activation function = "tanh"


In [18]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=5, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_1473


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 397 weights/biases, 15.7 KB, 3,718 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms                momentum    mean_weight            weight_rms          mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ----------------------  ----------  ---------------------  ------------------  ---------------------  --------------------
    1        64       Input    0.0
    2        5        Tanh     0.0        0.0   0.0   0.25239849831141326    0.4311668872833252      0.0         -0.015097625058115228  0.1779891848564148  -0.001059561762942135  0.12259435653686523
    3        5        Tanh     0.0        0.0   0.0   0.001980850868858397   0.000725434860214591    0.0         -0.06250577300786972   0.4883826971054077  0.06787601578286812    0.03093498945236206
    4        5        Tanh     0.0        0.0   0.0   0.011067370460368693   0.01314544677734375     0.0         0.17168970016762614    0.3957357406616211  0.05775672508541674    0.029345683753490448
    5        2        Softmax             0.0   0.0   0.0029493304900825024  0.00048607157077640295  0.0         1.6811195679008961     1.0491528511047363  9.71445146547012e-17   0.0905948281288147

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.01498420270229417
RMSE: 0.12240997795234737
LogLoss: 0.06761104157752912
Mean Per-Class Error: 0.013496616651956457
AUC: 0.9958811415122095
AUCPR: 0.9943769422396921
Gini: 0.991762283024419

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4126378161683046
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    202         4             0.0194   (4.0/206.0)
republican  1           131           0.0076   (1.0/132.0)
Total       203         135           0.0148   (5.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.412638     0.981273  96
max f2                       0.412638     0.987934  96
max f0point5                 0.750451     0.982704  87
max accuracy                 0.468988     0.985207  94
max precision                0.994877     1         0
max recall                   0.00511771   1         157
max specificity              0.994877     1         0
max absolute_mcc             0.412638     0.969218  96
max min_per_class_accuracy   0.468988     0.984848  94
max mean_per_class_accuracy  0.412638     0.986503  96
max tns                      0.994877     206       0
max fns                      0.994877     131       0
max fps                      0.00232277   206       269
max tps                      0.00511771   132       157
max tnr                      0.994877     1         0
max fnr                      0.994877     0.992424  0
max fpr                      0.00232277   1         269
max tpr                      0.00511771   1         157

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 38.22 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0118343                   0.99401            2.56061    2.56061            1                0.9

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 5, activation function = "tanh"


In [19]:
run_experiment(dataset=votes, features=features, target=target, hidden=[20,20,20], nfolds=5, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_1638


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 2,182 weights/biases, 37.2 KB, 3,718 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight             weight_rms           mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  ----------------------  -------------------  ---------------------  --------------------
    1        64       Input    0.0
    2        20       Tanh     0.0        0.0   0.0   0.252826455190916      0.4309566020965576     0.0         -0.0006878006061526776  0.15834683179855347  -0.002839801723190552  0.03326600790023804
    3        20       Tanh     0.0        0.0   0.0   0.004171751558678807   0.004089832305908203   0.0         0.005044844619114883    0.22238397598266602  0.008153478113334615   0.027608871459960938
    4        20       Tanh     0.0        0.0   0.0   0.004779363527632086   0.003946200013160706   0.0         0.015775103969644988    0.21445506811141968  0.004839899927885796   0.014566101133823395
    5        2        Softmax             0.0   0.0   0.0050722042447887365  0.0019112210720777512  0.0         0.03699496565386653     1.155776023864746    0.0                    0.010058801621198654

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.013591574998182653
RMSE: 0.11658291040363786
LogLoss: 0.04653735974662889
Mean Per-Class Error: 0.01106943218593704
AUC: 0.9989702853780523
AUCPR: 0.9983648341082199
Gini: 0.9979405707561047

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3079197538244427
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    203         3             0.0146   (3.0/206.0)
republican  1           131           0.0076   (1.0/132.0)
Total       204         134           0.0118   (4.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.30792      0.984962  95
max f2                       0.178935     0.990991  99
max f0point5                 0.332121     0.984848  93
max accuracy                 0.332121     0.988166  93
max precision                0.99998      1         0
max recall                   0.178935     1         99
max specificity              0.99998      1         0
max absolute_mcc             0.30792      0.975282  95
max min_per_class_accuracy   0.30792      0.985437  95
max mean_per_class_accuracy  0.30792      0.988931  95
max tns                      0.99998      206       0
max fns                      0.99998      131       0
max fps                      3.32585e-06  206       269
max tps                      0.178935     132       99
max tnr                      0.99998      1         0
max fnr                      0.99998      0.992424  0
max fpr                      3.32585e-06  1         269
max tpr                      0.178935     1         99

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 38.22 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0147929                   0.999939           2.56061    2.56061            1      

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 5, activation function = "tanh"

In [20]:
run_experiment(dataset=votes, features=features, target=target, hidden=[100,100,100], nfolds=5, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_1807


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 26,902 weights/biases, 329.7 KB, 3,042 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight            weight_rms           mean_bias               bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  --------------------  ----------  ---------------------  -------------------  ----------------------  ---------------------
    1        64       Input    0.0
    2        100      Tanh     0.0        0.0   0.0   0.2542380617823528    0.4302535057067871    0.0         0.002860984636457644   0.10962304472923279  -0.0019316008571227095  0.016101747751235962
    3        100      Tanh     0.0        0.0   0.0   0.016613428225019017  0.015794269740581512  0.0         0.0005696288145270046  0.1013302206993103   -0.0023456889666684296  0.015572823584079742
    4        100      Tanh     0.0        0.0   0.0   0.051580740910035096  0.1256159543991089    0.0         0.001125604475197906   0.10079056024551392  -0.001129116880542333   0.008223116397857666
    5        2        Softmax             0.0   0.0   0.005954488324932754  0.001837337389588356  0.0         -0.007474560034461319  0.5524005889892578   5.204170427930421e-18   0.0045544058084487915

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.014998880199205224
RMSE: 0.12246991548623369
LogLoss: 0.05048575285903044
Mean Per-Class Error: 0.015923801117975873
AUC: 0.9987496322447779
AUCPR: 0.9980920417232073
Gini: 0.9974992644895557

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.08438767889091867
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    201         5             0.0243   (5.0/206.0)
republican  1           131           0.0076   (1.0/132.0)
Total       202         136           0.0178   (6.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.0843877    0.977612  97
max f2                       0.0843877    0.986446  97
max f0point5                 0.820311     0.984375  88
max accuracy                 0.446828     0.982249  91
max precision                1            1         0
max recall                   0.0098061    1         105
max specificity              1            1         0
max absolute_mcc             0.0843877    0.963206  97
max min_per_class_accuracy   0.301779     0.980583  95
max mean_per_class_accuracy  0.0843877    0.984076  97
max tns                      1            206       0
max fns                      1            131       0
max fps                      3.78069e-10  206       269
max tps                      0.0098061    132       105
max tnr                      1            1         0
max fnr                      1            0.992424  0
max fpr                      3.78069e-10  1         269
max tpr                      0.0098061    1         105

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 38.33 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0118343                   0.999999           2.56061   2.56061            1           

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 10, activation function = "tanh"

In [21]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=10, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_1954


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 397 weights/biases, 15.7 KB, 3,718 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms                momentum    mean_weight            weight_rms          mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ----------------------  ----------  ---------------------  ------------------  ---------------------  --------------------
    1        64       Input    0.0
    2        5        Tanh     0.0        0.0   0.0   0.25239849831141326    0.4311668872833252      0.0         -0.015097625058115228  0.1779891848564148  -0.001059561762942135  0.12259435653686523
    3        5        Tanh     0.0        0.0   0.0   0.001980850868858397   0.000725434860214591    0.0         -0.06250577300786972   0.4883826971054077  0.06787601578286812    0.03093498945236206
    4        5        Tanh     0.0        0.0   0.0   0.011067370460368693   0.01314544677734375     0.0         0.17168970016762614    0.3957357406616211  0.05775672508541674    0.029345683753490448
    5        2        Softmax             0.0   0.0   0.0029493304900825024  0.00048607157077640295  0.0         1.6811195679008961     1.0491528511047363  9.71445146547012e-17   0.0905948281288147

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.01498420270229417
RMSE: 0.12240997795234737
LogLoss: 0.06761104157752912
Mean Per-Class Error: 0.013496616651956457
AUC: 0.9958811415122095
AUCPR: 0.9943769422396921
Gini: 0.991762283024419

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4126378161683046
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    202         4             0.0194   (4.0/206.0)
republican  1           131           0.0076   (1.0/132.0)
Total       203         135           0.0148   (5.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.412638     0.981273  96
max f2                       0.412638     0.987934  96
max f0point5                 0.750451     0.982704  87
max accuracy                 0.468988     0.985207  94
max precision                0.994877     1         0
max recall                   0.00511771   1         157
max specificity              0.994877     1         0
max absolute_mcc             0.412638     0.969218  96
max min_per_class_accuracy   0.468988     0.984848  94
max mean_per_class_accuracy  0.412638     0.986503  96
max tns                      0.994877     206       0
max fns                      0.994877     131       0
max fps                      0.00232277   206       269
max tps                      0.00511771   132       157
max tnr                      0.994877     1         0
max fnr                      0.994877     0.992424  0
max fpr                      0.00232277   1         269
max tpr                      0.00511771   1         157

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 38.22 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0118343                   0.99401            2.56061    2.56061            1                0.9

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 10, activation function = "tanh"


In [22]:
run_experiment(dataset=votes, features=features, target=target, hidden=[20,20,20], nfolds=10, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_2232


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 2,182 weights/biases, 37.2 KB, 3,718 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight             weight_rms           mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  ----------------------  -------------------  ---------------------  --------------------
    1        64       Input    0.0
    2        20       Tanh     0.0        0.0   0.0   0.252826455190916      0.4309566020965576     0.0         -0.0006878006061526776  0.15834683179855347  -0.002839801723190552  0.03326600790023804
    3        20       Tanh     0.0        0.0   0.0   0.004171751558678807   0.004089832305908203   0.0         0.005044844619114883    0.22238397598266602  0.008153478113334615   0.027608871459960938
    4        20       Tanh     0.0        0.0   0.0   0.004779363527632086   0.003946200013160706   0.0         0.015775103969644988    0.21445506811141968  0.004839899927885796   0.014566101133823395
    5        2        Softmax             0.0   0.0   0.0050722042447887365  0.0019112210720777512  0.0         0.03699496565386653     1.155776023864746    0.0                    0.010058801621198654

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.013591574998182653
RMSE: 0.11658291040363786
LogLoss: 0.04653735974662889
Mean Per-Class Error: 0.01106943218593704
AUC: 0.9989702853780523
AUCPR: 0.9983648341082199
Gini: 0.9979405707561047

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3079197538244427
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    203         3             0.0146   (3.0/206.0)
republican  1           131           0.0076   (1.0/132.0)
Total       204         134           0.0118   (4.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.30792      0.984962  95
max f2                       0.178935     0.990991  99
max f0point5                 0.332121     0.984848  93
max accuracy                 0.332121     0.988166  93
max precision                0.99998      1         0
max recall                   0.178935     1         99
max specificity              0.99998      1         0
max absolute_mcc             0.30792      0.975282  95
max min_per_class_accuracy   0.30792      0.985437  95
max mean_per_class_accuracy  0.30792      0.988931  95
max tns                      0.99998      206       0
max fns                      0.99998      131       0
max fps                      3.32585e-06  206       269
max tps                      0.178935     132       99
max tnr                      0.99998      1         0
max fnr                      0.99998      0.992424  0
max fpr                      3.32585e-06  1         269
max tpr                      0.178935     1         99

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 38.22 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0147929                   0.999939           2.56061    2.56061            1      

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 10, activation function = "tanh"


In [23]:
run_experiment(dataset=votes, features=features, target=target, hidden=[100,100,100], nfolds=10, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748263532696_2510


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 26,902 weights/biases, 329.7 KB, 3,380 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  --------------------  ----------  ---------------------  -------------------  -----------------------  ---------------------
    1        64       Input    0.0
    2        100      Tanh     0.0        0.0   0.0   0.25402592291752624   0.4303598403930664    0.0         0.002850416431990652   0.10965204238891602  -0.002121511415572303    0.01644880324602127
    3        100      Tanh     0.0        0.0   0.0   0.014729274890525267  0.015273809432983398  0.0         0.0005918479439063504  0.10139307379722595  -0.002265132596098386    0.016413040459156036
    4        100      Tanh     0.0        0.0   0.0   0.0467493204690516    0.12169978022575378   0.0         0.0011410521622886335  0.10082182288169861  -0.00010014555552796308  0.0070044007152318954
    5        2        Softmax             0.0   0.0   0.005079214095603675  0.001587945967912674  0.0         -0.007474577883258462  0.5522048473358154   2.6020852139652106e-18   0.0016043470241129398

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.01766353281088727
RMSE: 0.13290422420257103
LogLoss: 0.05895289242602224
Mean Per-Class Error: 0.007281553398058253
AUC: 0.9994115916446014
AUCPR: 0.9990783495028307
Gini: 0.9988231832892027

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7563318232968509
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    203         3             0.0146   (3.0/206.0)
republican  0           132           0        (0.0/132.0)
Total       203         135           0.0089   (3.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.756332     0.988764  96
max f2                       0.756332     0.995475  96
max f0point5                 0.985922     0.988924  86
max accuracy                 0.756332     0.991124  96
max precision                1            1         0
max recall                   0.756332     1         96
max specificity              1            1         0
max absolute_mcc             0.756332     0.9816    96
max min_per_class_accuracy   0.784491     0.985437  95
max mean_per_class_accuracy  0.756332     0.992718  96
max tns                      1            206       0
max fns                      1            131       0
max fps                      5.2036e-09   206       269
max tps                      0.756332     132       96
max tnr                      1            1         0
max fnr                      1            0.992424  0
max fpr                      5.2036e-09   1         269
max tpr                      0.756332     1         96

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 41.60 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0118343                   1                  2.56061  2.56061            1               

None


In [24]:
from tabulate import tabulate
from IPython.display import Markdown, display, HTML
headers = ["Activation Function", "Cross Validation", "Hidden Layers", "Training Error %", "Validation Error %", "Cross Validation Mean Error %", "Training Time"]
#print(metrics)
print(tabulate(mean_error_metrics, headers=headers, tablefmt="grid"))


+-----------------------+--------------------+-----------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [25]:
headers = ["Activation Function", "Cross Validation", "Hidden Layers", "Training Mean Square Error", "Validation Mean Square Error", "Cross Validation Mean Square Error"]

print(tabulate(mse_metrics, headers=headers, tablefmt="grid"))

+-----------------------+--------------------+-----------------+------------------------------+--------------------------------+--------------------------------------+
| Activation Function   | Cross Validation   | Hidden Layers   |   Training Mean Square Error |   Validation Mean Square Error | Cross Validation Mean Square Error   |
+=======================+====================+=================+==============================+================================+======================================+
| rectifier             | None               | [5, 5, 5]       |                    0.0248412 |                      0.0322698 | -                                    |
+-----------------------+--------------------+-----------------+------------------------------+--------------------------------+--------------------------------------+
| rectifier             | None               | [20, 20, 20]    |                    0.0145144 |                      0.0254836 | -                              

In [26]:
headers = ["Activation Function", "Cross Validation", "Hidden Layers", "Training Root Mean Square Error", "Validation Root Mean Square Error", "Cross Validation Root Mean Square Error"]

print(tabulate(rmse_metrics, headers=headers, tablefmt="grid"))

+-----------------------+--------------------+-----------------+-----------------------------------+-------------------------------------+-------------------------------------------+
| Activation Function   | Cross Validation   | Hidden Layers   |   Training Root Mean Square Error |   Validation Root Mean Square Error | Cross Validation Root Mean Square Error   |
+=======================+====================+=================+===================================+=====================================+===========================================+
| rectifier             | None               | [5, 5, 5]       |                          0.157611 |                            0.179638 | -                                         |
+-----------------------+--------------------+-----------------+-----------------------------------+-------------------------------------+-------------------------------------------+
| rectifier             | None               | [20, 20, 20]    |                     